In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [24]:
class make_partitions:
    def make_parts(self,df):
        def make_parti(df,partitions,stratify_columns,independent_columns=None,random_seed=None):

            np.random.seed(random_seed)

            if isinstance(partitions, int):
                partitions = [1.0 / partitions for p in range(partitions - 1)]
            elif sum(partitions) >= 1:
                raise Exception('Partitions proportions must sum less than 1')
            partitions.append(1 - sum(partitions))

            n_sets = len(partitions)

            partidx = {i: [] for i in range(n_sets)}

            if independent_columns is not None:
                groups = df.groupby(stratify_columns).groups.items()
                dist = {k: len(v) * np.array(partitions) for k, v in groups}
                part = {k: np.zeros(n_sets) for k, v in groups}

                for k, g in tqdm(df.groupby(independent_columns)):
                    diffs = []
                    for i, r in g.iterrows():
                        group = tuple(r[stratify_columns].to_list())
                        diffs.append(dist[group] - part[group])
                    pix = np.max(np.array(diffs), 0).argmax()
                    gix = np.max(np.array(diffs), 1).argmax()
                    partidx[pix].extend(g.index)
                    group = tuple(g.iloc[gix][stratify_columns].to_list())
                    part[group][pix] += len(g)
            else:
                for k, g in df.groupby(stratify_columns):
                    ix = np.tile(np.arange(n_sets),len(g)//n_sets)
                    ix = np.hstack([ix,np.random.choice(np.arange(n_sets),len(g)-len(ix), replace=False)])
        #             ix = np.random.permutation(
        #                 sum([[i] * int(np.ceil(len(g) * p))
        #                      for i, p in enumerate(partitions)], []))[:len(g)]
                    for i in range(n_sets):
                        partidx[i].extend(g[ix == i].index)
            return partidx
    
        df_=pd.DataFrame()
            
        df=df.sample(frac=1)
        
        nquant = 4
        boundaries = df['labels_mean'].quantile(np.linspace(0,1,nquant))
        df['labels_mean_quant'] = np.searchsorted(boundaries, df['labels_mean'])

        pix = make_parti(df,5, ['ethnicity','gender','music','labels_mean_quant'], independent_columns='basename')

        for k,v in pix.items():
            df.loc[v,'partition'] = int(k)
            #df.loc[v,'group'] = int(i)
        df_=df_.append(df)
        
        return df_

In [2]:
def make_strat_folds(df,random_seed=None):

        df_=pd.DataFrame()
        
        df=df.sample(frac=1)
        partitions=5
        nquant = 4
        boundaries = df['labels_mean'].quantile(np.linspace(0,1,nquant))
        df['labels_mean_quant'] = np.searchsorted(boundaries, df['labels_mean'])        

        np.random.seed(random_seed)

        stratify_columns=['ethnicity','gender','music','labels_mean_quant']
        independent_columns='basename'

        if isinstance(partitions, int):
            partitions = [1.0 / partitions for p in range(partitions - 1)]
        elif sum(partitions) >= 1:
            raise Exception('Partitions proportions must sum less than 1')
        partitions.append(1 - sum(partitions))

        n_sets = len(partitions)

        partidx = {i: [] for i in range(n_sets)}

        if independent_columns is not None:
            groups = df.groupby(stratify_columns).groups.items()
            dist = {k: len(v) * np.array(partitions) for k, v in groups}
            part = {k: np.zeros(n_sets) for k, v in groups}

            for k, g in tqdm(df.groupby(independent_columns)):
                diffs = []
                for i, r in g.iterrows():
                    group = tuple(r[stratify_columns].to_list())
                    diffs.append(dist[group] - part[group])
                pix = np.max(np.array(diffs), 0).argmax()
                gix = np.max(np.array(diffs), 1).argmax()
                partidx[pix].extend(g.index)
                group = tuple(g.iloc[gix][stratify_columns].to_list())
                part[group][pix] += len(g)
        else:
            for k, g in df.groupby(stratify_columns):
                ix = np.tile(np.arange(n_sets),len(g)//n_sets)
                ix = np.hstack([ix,np.random.choice(np.arange(n_sets),len(g)-len(ix), replace=False)])
    #             ix = np.random.permutation(
    #                 sum([[i] * int(np.ceil(len(g) * p))
    #                      for i, p in enumerate(partitions)], []))[:len(g)]
                for i in range(n_sets):
                    partidx[i].extend(g[ix == i].index)        

        pix = partidx

        for k,v in pix.items():
            df.loc[v,'fold'] = int(k)
            #df.loc[v,'group'] = int(i)
        df_=df_.append(df)
        
        return df_

In [25]:
a=make_partitions()

In [10]:
make_strat_folds(df)

  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_133961/2157400905.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


,filename,basename,extraversion,neuroticism,agreeableness,conscientiousness,openness,labels_mean,yamnet_no_music_20+speech_rate_>0.5,yamnet_no_music_20+speech_rate_>0.6,...,silero_speech_rate>0.5,silero_speech_rate>0.6,silero_speech_rate>0.7,silero_speech_rate>0.8,silero_speech_rate>0.9,ethnicity,gender,music,labels_mean_quant,partition
6833,2KSBoJZMcMU.004.wav,2KSBoJZMcMU,0.429907,0.458333,0.428571,0.514563,0.655556,0.497386,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,2,1,0.0,2,3.0
6425,1YsZ4GPpyBA.004.wav,1YsZ4GPpyBA,0.476636,0.677083,0.549451,0.582524,0.677778,0.592694,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,2,2,0.0,3,3.0
4977,kDXYolv-8p8.002.wav,kDXYolv-8p8,0.439252,0.572917,0.648352,0.456311,0.611111,0.545588,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,2,1,1.0,2,1.0
967,uwYQKM_STCw.001.wav,uwYQKM_STCw,0.542056,0.416667,0.450549,0.417476,0.588889,0.483127,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2,2,0.0,2,1.0
3694,CFK8ib0aWe8.000.wav,CFK8ib0aWe8,0.411215,0.489583,0.483516,0.601942,0.500000,0.497251,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2,2,0.0,2,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6396,dmOKZLeFKCM.002.wav,dmOKZLeFKCM,0.803738,0.697917,0.615385,0.844660,0.600000,0.712340,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,2,2,0.0,3,4.0
768,VugKq4gOi50.001.wav,VugKq4gOi50,0.551402,0.593750,0.670330,0.679612,0.722222,0.643463,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2,1,0.0,3,0.0
6985,gq0yaEv6gSE.000.wav,gq0yaEv6gSE,0.261682,0.260417,0.296703,0.320388,0.255556,0.278949,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,2,1,0.0,1,3.0
9148,BStHNKV820Q.002.wav,BStHNKV820Q,0.626168,0.708333,0.582418,0.718447,0.600000,0.647073,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,2,2,0.0,3,3.0


In [9]:
df = pd.read_csv('/home/gbarchi/Documentos/Trust/OCEAN-TRUST/data/metadata/stratified_df.csv')

## 20 groups - 5 folds (all samples)

In [24]:
n_groups=20
df_=make_parts(n_groups,df)
df_.to_csv('first_impressions_parts-20_groups-5_folds.csv')

  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

...

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


In [25]:
df_

,filename,basename,extraversion,neuroticism,agreeableness,conscientiousness,openness,labels_mean,yamnet_no_music_20+speech_rate_>0.5,yamnet_no_music_20+speech_rate_>0.6,...,silero_speech_rate>0.6,silero_speech_rate>0.7,silero_speech_rate>0.8,silero_speech_rate>0.9,ethnicity,gender,music,labels_mean_quant,partition,group
7077,o_kTWEEjtyw.001.wav,o_kTWEEjtyw,0.280374,0.302083,0.439560,0.359223,0.488889,0.374026,1.0,1.0,...,1.0,1.0,1.0,0.0,2,1,0.0,1,0.0,0.0
8099,4vdJGgZpj4k.000.wav,4vdJGgZpj4k,0.448598,0.572917,0.582418,0.553398,0.577778,0.547022,0.0,0.0,...,1.0,1.0,0.0,0.0,2,1,1.0,2,1.0,0.0
3940,puJ_NbKzRTE.003.wav,puJ_NbKzRTE,0.439252,0.489583,0.461538,0.398058,0.588889,0.475464,1.0,1.0,...,1.0,1.0,1.0,1.0,2,2,0.0,1,1.0,0.0
1151,zyGz_H1UTnQ.002.wav,zyGz_H1UTnQ,0.299065,0.427083,0.472527,0.786408,0.444444,0.485906,1.0,1.0,...,1.0,1.0,1.0,0.0,2,1,0.0,2,1.0,0.0
8580,DfpKVl9Zpdg.000.wav,DfpKVl9Zpdg,0.299065,0.218750,0.296703,0.281553,0.311111,0.281437,1.0,1.0,...,1.0,1.0,0.0,0.0,3,1,0.0,1,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,Gdrgw7Z6tLg.002.wav,Gdrgw7Z6tLg,0.485981,0.583333,0.626374,0.339806,0.600000,0.527099,1.0,1.0,...,1.0,1.0,1.0,1.0,2,1,0.0,2,2.0,19.0
1220,Ssy7T1T0yKg.003.wav,Ssy7T1T0yKg,0.383178,0.187500,0.373626,0.281553,0.366667,0.318505,0.0,0.0,...,1.0,1.0,1.0,0.0,2,1,1.0,1,3.0,19.0
2585,Gg4OMtVe7lk.002.wav,Gg4OMtVe7lk,0.785047,0.760417,0.747253,0.504854,0.800000,0.719514,1.0,1.0,...,1.0,1.0,1.0,1.0,2,1,0.0,3,3.0,19.0
2376,xQHmbJ_lPjY.001.wav,xQHmbJ_lPjY,0.560748,0.541667,0.670330,0.582524,0.600000,0.591054,1.0,1.0,...,1.0,1.0,1.0,1.0,2,2,0.0,3,1.0,19.0


## 200 groups - 20 folds

In [27]:
n_groups=200
df_=make_parts(n_groups,df)
df_.to_csv('first_impressions_parts-200_groups-5_folds.csv')

  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

...

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/3060 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


## Non-speech Subset 

In [28]:
subset='/home/gbarchi/Documentos/Trust/OCEAN-TRUST/data/lists/silero_th_0.5_no_speech.txt'

In [29]:
with open (subset) as file:
    subset_list=file.read().splitlines()

In [31]:
df_subset=df[df['filename'].isin(subset_list)]

In [32]:
df_subset

,filename,basename,extraversion,neuroticism,agreeableness,conscientiousness,openness,labels_mean,yamnet_no_music_20+speech_rate_>0.5,yamnet_no_music_20+speech_rate_>0.6,...,yamnet_no_music_20+speech_rate_>0.8,yamnet_no_music_20+speech_rate_>0.9,silero_speech_rate>0.5,silero_speech_rate>0.6,silero_speech_rate>0.7,silero_speech_rate>0.8,silero_speech_rate>0.9,ethnicity,gender,music
0,mDRMCwSD3Ms.001.wav,mDRMCwSD3Ms,0.579439,0.697917,0.736264,0.766990,0.588889,0.673900,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2,1,1.0
3,XB81sertsYU.004.wav,XB81sertsYU,0.588785,0.510417,0.648352,0.466019,0.666667,0.576048,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2,2,1.0
13,51KRxB3g7A8.001.wav,51KRxB3g7A8,0.644860,0.656250,0.648352,0.699029,0.622222,0.654143,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2,2,0.0
21,X1EjfXX3NTk.002.wav,X1EjfXX3NTk,0.289720,0.437500,0.461538,0.466019,0.488889,0.428733,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2,1,1.0
23,4CiR99jTzso.002.wav,4CiR99jTzso,0.495327,0.510417,0.582418,0.398058,0.577778,0.512799,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9943,U2RemWUyjts.002.wav,U2RemWUyjts,0.327103,0.375000,0.582418,0.485437,0.488889,0.451769,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,0.0
9955,J28DwJsK8Do.001.wav,J28DwJsK8Do,0.738318,0.739583,0.681319,0.786408,0.722222,0.733570,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0.0
9977,hhdoeK8SsiI.004.wav,hhdoeK8SsiI,0.551402,0.447917,0.571429,0.485437,0.600000,0.531237,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,1.0
9984,PReOtefm17s.004.wav,PReOtefm17s,0.542056,0.656250,0.560440,0.650485,0.800000,0.641846,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,1.0


Subset 200 groups - 5 folds

In [33]:
n_groups=200
df_=make_parts(n_groups,df_subset)
df_.to_csv('first_impressions_parts-no_speech_subset-200_groups-5_folds.csv')

  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

...

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


20 groups - 5 Folds

In [34]:
n_groups=20
df_=make_parts(n_groups,df_subset)
df_.to_csv('first_impressions_parts-no_speech_subset-20_groups-5_folds.csv')

  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

...

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)


  0%|          | 0/988 [00:00<?, ?it/s]

/tmp/ipykernel_315483/3802422626.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_=df_.append(df)
